In [226]:
import h5py
import scanpy as sc
import anndata
import loompy as lp

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sys
import glob
import importlib

In [227]:
import sys
sys.path.insert(0, '/home/tchari/monod/src/')

In [228]:
import monod
importlib.reload(monod)
from monod import preprocess, extract_data, cme_toolbox, analysis, mminference

In [229]:
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score
from sklearn.cluster import KMeans
from sklearn.neighbors import kneighbors_graph

import igraph as ig
import leidenalg as la

In [230]:
#Perform standard normalization (read-depth + log1p)
def norm_mat(mat,new_tot=1e4):
    '''
    mat: cell x gene matrix
    '''  
    tots = np.sum(mat,axis=1)
    divids = (new_tot/tots)[:,None]
    mat_norm = mat*divids
    mat_norm = np.log1p(mat_norm)
    
    mat_norm[np.isnan(mat_norm)] = 0
    
    return mat_norm

In [231]:
def get_graph(mat,neigh=30):
    A = kneighbors_graph(mat, neigh, mode='connectivity', include_self=True)
    sources, targets = A.nonzero()
    weights = A[sources, targets]
    if isinstance(weights, np.matrix):
        weights = weights.A1
    g = ig.Graph(directed=False) #True in scanpy, which is odd
    g.add_vertices(A.shape[0])  # this adds adjacency.shape[0] vertices
    g.add_edges(list(zip(sources, targets)))

    g.es['weight'] = weights
    
    return g

In [5]:
# ********* Get meK_looms.tar.gz and meKMeans_data_samp_params.csv *********

samps = pd.read_csv('meKMeans_data_samp_params.csv')
samps.head()

,Dataset,Transcrip,Clusters,Cu,Lam
0,allen_b08,/home/tchari/perturbCME/notebooks/gg_200524_mo...,10,-6.777778,-1.25
1,allen_b02h01a02,/home/tchari/perturbCME/notebooks/gg_200524_mo...,11,-6.222222,-1.25
2,cl3,/home/tchari/perturbCME/notebooks/gg_200525_ge...,3,-6.777778,-0.70
3,cl5,/home/tchari/perturbCME/notebooks/gg_200525_ge...,5,-7.888889,-1.25
4,brca1,/home/tchari/perturbCME/notebooks/gg_200524_mo...,-1,-6.777778,-0.70


In [6]:
d_strings  = glob.glob('./hvg_objs_0215/*.loom')
d_strings 

['./hvg_objs_0215/cl5_357hvgs.loom',
 './hvg_objs_0215/cl5_1193all.loom',
 './hvg_objs_0215/allen_b02h01a02_359hvgs.loom',
 './hvg_objs_0215/cl3_466hvgs.loom',
 './hvg_objs_0215/allen_b08_682hvgs.loom',
 './hvg_objs_0215/allen_b02h01a02_2770all.loom',
 './hvg_objs_0215/cl3_1137all.loom',
 './hvg_objs_0215/allen_b08_1948all.loom']

In [7]:
datas = pd.unique(samps.Dataset)
datas

array(['allen_b08', 'allen_b02h01a02', 'cl3', 'cl5', 'brca1', 'pbmc',
       'e11e13'], dtype=object)

In [8]:
attribute_names =[('unspliced','spliced'),'gene_name','barcode']

### **meK-Means**

Test with 2, correct K, K+5, K+10

In [24]:
# !rm -r /home/tchari/perturbCME/notebooks/fits/meKruns_0209

In [9]:
#For each loom in looms from ./hvg_objs

# !mkdir /home/tchari/perturbCME/notebooks/fits/meKruns_0209
!mkdir /home/tchari/perturbCME/notebooks/fits/meKruns_0216


dir_strings = []
data_strings = []
result_strings = []

#Save method, clustering, mat_in (U,S, etc), hyper (K or res), hvgs, data, ARI, AMI
meK_res = pd.DataFrame()
meths = []
result = []
mats = []
hyper = []
hvgs = []
data = []
ari = []
ami = []

for d in d_strings:
    
    #Get tech param by keyword (datas) in loom paths
    ind = [i in d for i in datas]
    
    #Inference params for sampling
    tech_tup = [list(samps.Cu[ind])[0],list(samps.Lam[ind])[0]]
    
    #Preprocessing params for filepaths
    trans = list(samps.Transcrip[ind])[0]
    k = list(samps.Clusters[ind])[0]
    data_name = list(samps.Dataset[ind])[0]
    
    
    ds = lp.connect(d)
    g_names = ds.ra[attribute_names[1]]
    num_genes = len(g_names)
    num_cells = len(ds.ca[attribute_names[2]])
    
    U = ds.layers['unspliced'][:,:].T
    S = ds.layers['spliced'][:,:].T


    true_labs = list(ds.ca['subclass_label'])
    ds.close()
    print(data_name+' with '+str(num_genes)+' genes')
    
#     #Try filter
#     var_threshold = 1.5
#     U_mean = U.mean(0)
#     S_mean = S.mean(0)
#     U_var = U.var(0)
#     S_var = S.var(0)
#     fitted_idx = (U_mean > 0.02) & (S_mean > 0.02) \
#         & (((U_var-U_mean)/(U_mean**2)) > var_threshold)\
#         & (((S_var-S_mean)/(S_mean**2)) > var_threshold)\
#         & (np.abs(np.log(S_mean/U_mean)) < 4)  #******* FLip top condition for nuclear? ********
    
#     markers = g_names[fitted_idx]
#     print('Markers: ',len(markers))
#     print('Mean U/S: ', np.mean(U_mean/S_mean))
    
    Ks = [k,k+5] #2,

    for clus in Ks:
       
        #Run Monod preprocessing to set up files for run, leave room for genes to be removed with low U/S
        dir_string,dataset_string = monod.preprocess.construct_batch([d], \
                                                     trans, \
                                                     [data_name], \
                                                     attribute_names=attribute_names,\
                                                     batch_location='./fits/meKruns_0216', \
                                                     meta=data_name+'_K'+str(clus), \
                                                     batch_id=1, \
                                                     n_genes=num_genes,exp_filter_threshold=None,
                                                                     viz=False,genes_to_fit=g_names)
                                                        #num_genes-60 or 100,genes_to_fit=markers
        dir_strings += [dir_string]
        data_strings += [dataset_string]

        #Set model and run meK-Means
        
        #Define bounds for params
        phys_lb = [-2.0, -1.8, -1.8 ] 
        phys_ub = [4.2, 2.5, 2.5] 
        samp_lb = tech_tup 
        samp_ub = tech_tup  
        gridsize = [1,1] #Already have tech params
    

        epochs = 10

        # ---------------- meK-Means Inference ---------------- 
        #Define model with bursty transcription and Poisson molecule capture/sampling
        fitmodel = monod.cme_toolbox.CMEModel('Bursty','Poisson')

        #Set up mminference parameters
        inference_parameters = monod.mminference.InferenceParameters(phys_lb,phys_ub,samp_lb,samp_ub,gridsize,\
                    dataset_string[0],fitmodel,k=clus,epochs=epochs,use_lengths = True,
                    gradient_params = {'max_iterations':5,'init_pattern':'moments','num_restarts':1})

        #Read in loom file with filtered barcodes
        search_data = monod.extract_data.extract_data(d, trans, data_name,
                    dataset_string[0], dir_string, viz=False, dataset_attr_names=attribute_names)

        #Run inference(fit_all_grid_points()) and Save result file strings
        full_result_string = inference_parameters.fit_all_grid_points(40,search_data) 

        result_strings.append(full_result_string)
        

        # ----------------  Save output ---------------- 
        #Read in results and get cluster assignments
        sr = [monod.analysis.load_search_results(i) for i in full_result_string]
        
        
        cat_assigns = np.array([None]*num_cells) #cat_assigns is empty list of length = no. cell barcodes
        for i in range(len(sr)):
            r = sr[i]
            cat_assigns[r.filt] = r.assigns #denotes which cluster (k), cells were assigned to

        print('No. clus: ',len(np.unique(cat_assigns)))
        

        meths += ['meK-Means']
        result += [cat_assigns]
        mats += ['U,S']
        hyper += [clus]
        hvgs += [num_genes]
        data += [data_name]
        ari += [adjusted_rand_score(true_labs,cat_assigns)]
        ami += [adjusted_mutual_info_score(true_labs,cat_assigns)]
        
        print('ARI: ',adjusted_rand_score(true_labs,cat_assigns))


meK_res['Method'] = meths
meK_res['Clustering'] = result
meK_res['Matrices'] = mats
meK_res['Hyperparam'] = hyper
meK_res['Genes'] = hvgs
meK_res['Data'] = data
meK_res['ARI'] = ari
meK_res['AMI'] = ami
    

cl5 with 357 genes


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:14<00:00,  2.91s/it]


mstep self.weights:  [0.22184715 0.31965643 0.19363111 0.11203402 0.1528313 ]
Q Function:  -1613996.7060211934



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:14<00:00,  2.90s/it]


mstep self.weights:  [0.2215189  0.3205985  0.19456084 0.1116819  0.15163985]
Q Function:  -1582351.6245810064



...

mstep self.weights:  [5.48041455e-02 5.43951941e-02 1.92175855e-01 9.91236555e-02
 1.30369789e-01 4.42648868e-02 1.75654312e-10 6.23516612e-02
 4.34180894e-02 9.81565175e-02 1.51413273e-01 3.39012709e-02
 1.06827484e-02 1.75654312e-10 2.49429125e-02]
Q Function:  -11287736.420727748



100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [05:26<00:00, 27.20s/it]


mstep self.weights:  [5.48041455e-02 5.47986953e-02 1.96361674e-01 9.98697551e-02
 1.30527659e-01 4.42648868e-02 1.75654312e-10 6.27159431e-02
 4.52500414e-02 9.81742533e-02 1.54388752e-01 3.39012817e-02
 1.75654312e-10 1.75654312e-10 2.49429125e-02]
Q Function:  -11298717.76842861



100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [05:29<00:00, 27.42s/it]


mstep self.weights:  [5.48041455e-02 5.51832049e-02 1.90503455e-01 1.01102941e-01
 1.30889746e-01 4.42648868e-02 1.75654312e-10 6.50175261e-02
 4.51982147e-02 9.78651242e-02 1.56326560e-01 3.39012824e-02
 1.75654312e-10 1.75654312e-10 2.49429125e-02]
Q Function:  -11311125.864565369

No. clus:  12
ARI:  0.44747204805875873


In [10]:
meK_res.to_pickle('./fits/meKruns_0216/meK_res_filt.pkl')
#unpickled_df = pd.read_pickle("./meK_res.pkl")

In [11]:
unpickled_df = pd.read_pickle('./fits/meKruns_0216/meK_res_filt.pkl')

In [12]:
unpickled_df

,Method,Clustering,Matrices,Hyperparam,Genes,Data,ARI,AMI
0,meK-Means,"[4, 4, 0, 4, 4, 4, 3, 3, 4, 4, 0, 0, 4, 4, 4, ...","U,S",5,357,cl5,0.982402,0.968188
1,meK-Means,"[3, 3, 2, 3, 3, 3, 4, 4, 3, 3, 2, 2, 3, 3, 3, ...","U,S",10,357,cl5,0.935710,0.932768
2,meK-Means,"[1, 1, 0, 1, 1, 1, 4, 4, 1, 1, 0, 0, 1, 1, 1, ...","U,S",5,1193,cl5,0.983810,0.971894
3,meK-Means,"[5, 5, 3, 5, 5, 5, 0, 0, 5, 5, 3, 3, 5, 5, 5, ...","U,S",10,1193,cl5,0.928421,0.928520
4,meK-Means,"[5, 3, 3, 8, 7, 1, 6, 3, 7, 8, 1, 0, 1, 7, 3, ...","U,S",11,359,allen_b02h01a02,0.666748,0.802591
5,meK-Means,"[15, 2, 2, 1, 3, 0, 11, 2, 3, 1, 0, 12, 0, 4, ...","U,S",16,359,allen_b02h01a02,0.522266,0.782716
6,meK-Means,"[2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 1, ...","U,S",3,466,cl3,0.986588,0.974199
7,meK-Means,"[7, 7, 7, 7, 7, 3, 7, 7, 7, 7, 3, 7, 7, 7, 3, ...","U,S",8,466,cl3,0.986588,0.974199
8,meK-Means,"[0, 2, 5, 0, 3, 8, 3, 4, 2, 8, 3, 7, 0, 2, 2, ...","U,S",10,682,allen_b08,0.561534,0.776994
9,meK-Means,"[0, 8, 5, 13, 1, 11, 1, 12, 6, 13, 1, 9, 13, 8...","U,S",15,682,allen_b08,0.452264,0.757664


In [15]:
unpickled_df

,Method,Clustering,Matrices,Hyperparam,Genes,Data,ARI,AMI
0,meK-Means,"[4, 4, 3, 4, 4, 4, 0, 0, 4, 4, 3, 3, 4, 4, 4, ...","U,S",5,357,cl5,0.982644,0.967017
1,meK-Means,"[8, 8, 6, 8, 8, 8, 3, 3, 8, 8, 6, 6, 8, 8, 8, ...","U,S",10,357,cl5,0.980424,0.965182
2,meK-Means,"[3, 3, 2, 3, 3, 3, 0, 0, 3, 3, 2, 2, 3, 3, 3, ...","U,S",5,1193,cl5,0.982576,0.969364
3,meK-Means,"[0, 0, 1, 0, 0, 0, 6, 6, 0, 0, 1, 1, 0, 0, 0, ...","U,S",10,1193,cl5,0.980907,0.966469
4,meK-Means,"[6, 5, 5, 8, 4, 3, 7, 5, 4, 8, 3, 0, 0, 4, 5, ...","U,S",11,359,allen_b02h01a02,0.517851,0.753212
5,meK-Means,"[2, 4, 4, 7, 15, 5, 6, 4, 13, 7, 5, 10, 5, 3, ...","U,S",16,359,allen_b02h01a02,0.496064,0.753997
6,meK-Means,"[2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0, 2, 2, 2, 0, ...","U,S",3,466,cl3,0.989820,0.980961
7,meK-Means,"[6, 6, 6, 6, 6, 5, 6, 6, 6, 6, 5, 6, 6, 6, 4, ...","U,S",8,466,cl3,0.777249,0.831015
8,meK-Means,"[1, 4, 6, 1, 0, 9, 0, 3, 4, 9, 0, 5, 1, 0, 4, ...","U,S",10,682,allen_b08,0.557244,0.773200
9,meK-Means,"[13, 5, 3, 9, 0, 6, 7, 12, 11, 2, 7, 4, 9, 5, ...","U,S",15,682,allen_b08,0.481304,0.767250


### **Leiden & K-Means**


In [28]:
# ! rm -r /home/tchari/perturbCME/notebooks/fits/LeidKMeansruns_0209

In [18]:
#Make the U,S, U+S, Concat matrices, res 0.75,1,1.5,2

!mkdir /home/tchari/perturbCME/notebooks/fits/LeidKMeansruns_0215


#Save method, clustering, mat_in (U,S, etc), hyper (K or res), hvgs, data, ARI, AMI
LK_res = pd.DataFrame()
meths = []
result = []
mats = []
hyper = []
hvgs = []
data = []
ari = []
ami = []

for d in d_strings:
    
    #Get tech param by keyword (datas) in loom paths
    ind = [i in d for i in datas]
    
    k = list(samps.Clusters[ind])[0]
    data_name = list(samps.Dataset[ind])[0]
    
    
    ds = lp.connect(d)
    num_genes = len(ds.ra[attribute_names[1]])
    num_cells = len(ds.ca[attribute_names[2]])
    U = ds.layers[attribute_names[0][0]][:,:].T
    S = ds.layers[attribute_names[0][1]][:,:].T
    true_labs = list(ds.ca['subclass_label'])
    ds.close()
    print(data_name+' with '+str(num_genes)+' HVGs')
    
    res = [0.75,1,1.5,2]
    
    #Get U,S,U_p_S,U_c_S
        
    #Norm mats
    U_norm = norm_mat(U)
    S_norm = norm_mat(S)
    U_p_S_norm = norm_mat(U+S)
    U_c_S_norm = norm_mat(np.concatenate([U,S],axis=1))
    
    g_U = get_graph(U_norm)
    g_S = get_graph(S_norm)
    g_U_p_S = get_graph(U_p_S_norm)
    g_U_c_S = get_graph(U_c_S_norm)
    
    # -------- Fit Leiden -------
    for r in res:
        
        partition_type = la.RBConfigurationVertexPartition
        #Make Knn graphs
        for g in [g_U,g_S,g_U_p_S,g_U_c_S]:
            part = la.find_partition(g, partition_type, 
                                        weights=np.array(g.es['weight']).astype(np.float64), 
                                        n_iterations=-1, resolution_parameter=r)

            labels = np.array(part.membership)
            
            # ----- Save Results ---- ARI,AMI etc
            meths += ['Leiden']
            result += [labels]
            hyper += [r]
            hvgs += [num_genes]
            data += [data_name]
            ari += [adjusted_rand_score(true_labs,labels)]
            ami += [adjusted_mutual_info_score(true_labs,labels)]
            
#             print('No. clus Leiden: ',len(np.unique(labels)))
            
        mats += ['U','S','U+S','UcatS']
        
        
        
    # -------- Fit K-Means with k -------
    
    for x in [U_norm,S_norm,U_p_S_norm,U_c_S_norm]:
        kmeans = KMeans(n_clusters=k, random_state=0).fit(x)
        labs = kmeans.labels_
        
        meths += ['K-Means']
        result += [labs]
        hyper += [k]
        hvgs += [num_genes]
        data += [data_name]
        ari += [adjusted_rand_score(true_labs,labs)]
        ami += [adjusted_mutual_info_score(true_labs,labs)]
        
#         print('No. clus K-Means: ',len(np.unique(labs)))
        
    mats += ['U','S','U+S','UcatS']

    
LK_res['Method'] = meths
LK_res['Clustering'] = result
LK_res['Matrices'] = mats
LK_res['Hyperparam'] = hyper
LK_res['HVGs'] = hvgs
LK_res['Data'] = data
LK_res['ARI'] = ari
LK_res['AMI'] = ami

cl5 with 357 HVGs


/home/tchari/.local/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in true_divide
  import sys
/home/tchari/.local/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in multiply
  


cl5 with 1193 HVGs
allen_b02h01a02 with 359 HVGs
cl3 with 466 HVGs
allen_b08 with 682 HVGs
allen_b02h01a02 with 2770 HVGs
cl3 with 1137 HVGs
allen_b08 with 1948 HVGs


In [ ]:
#Save method, clustering, mat_in (U,S, etc), hyper (K or res), hvgs, data, ARI, NMI
# LK_res.to_csv('./fits/LeidKMeansruns_0209/LeidKMeans_bench_results.csv',index=None)
# LK_res.head()



In [19]:
LK_res.to_pickle('./fits/LeidKMeansruns_0215/LK_res.pkl')
#unpickled_df = pd.read_pickle("./meK_res.pkl")

In [13]:
unpickled_df_LK = pd.read_pickle("./fits/LeidKMeansruns_0215/LK_res.pkl")
unpickled_df_LK

,Method,Clustering,Matrices,Hyperparam,HVGs,Data,ARI,AMI
0,Leiden,"[3, 3, 1, 3, 3, 3, 4, 4, 3, 3, 1, 1, 3, 3, 3, ...",U,0.75,357,cl5,0.948212,0.915683
1,Leiden,"[3, 3, 1, 3, 3, 3, 4, 4, 3, 3, 1, 1, 3, 3, 3, ...",S,0.75,357,cl5,0.986832,0.975747
2,Leiden,"[3, 3, 1, 3, 3, 3, 4, 4, 3, 3, 1, 1, 3, 3, 3, ...",U+S,0.75,357,cl5,0.965742,0.946763
3,Leiden,"[3, 3, 1, 3, 3, 3, 4, 4, 3, 3, 1, 1, 3, 3, 3, ...",UcatS,0.75,357,cl5,0.965702,0.947080
4,Leiden,"[3, 3, 1, 3, 3, 3, 4, 4, 3, 3, 1, 1, 3, 3, 3, ...",U,1.00,357,cl5,0.948656,0.916597
...,...,...,...,...,...,...,...,...
155,Leiden,"[4, 1, 9, 4, 0, 10, 2, 11, 1, 13, 7, 2, 3, 1, ...",UcatS,2.00,1948,allen_b08,0.490543,0.789872
156,K-Means,"[9, 0, 2, 9, 3, 8, 6, 2, 0, 1, 3, 6, 1, 0, 0, ...",U,10.00,1948,allen_b08,0.449502,0.734600
157,K-Means,"[0, 1, 5, 0, 4, 6, 2, 3, 1, 6, 4, 2, 7, 1, 1, ...",S,10.00,1948,allen_b08,0.583041,0.798872
158,K-Means,"[2, 0, 3, 2, 1, 8, 6, 3, 0, 8, 1, 6, 5, 0, 0, ...",U+S,10.00,1948,allen_b08,0.572679,0.786544


In [21]:
unpickled_df_LK[unpickled_df_LK.Method.isin(['K-Means']) ]

,Method,Clustering,Matrices,Hyperparam,HVGs,Data,ARI,AMI
16,K-Means,"[0, 0, 1, 0, 0, 0, 3, 3, 0, 0, 1, 1, 0, 0, 0, ...",U,5.0,357,cl5,0.961551,0.935268
17,K-Means,"[0, 0, 4, 0, 0, 0, 3, 3, 0, 0, 4, 4, 0, 0, 0, ...",S,5.0,357,cl5,0.985684,0.971379
18,K-Means,"[0, 0, 2, 0, 0, 0, 4, 4, 0, 0, 2, 2, 0, 0, 0, ...",U+S,5.0,357,cl5,0.986565,0.975978
19,K-Means,"[0, 0, 1, 0, 0, 0, 4, 4, 0, 0, 1, 1, 0, 0, 0, ...",UcatS,5.0,357,cl5,0.984452,0.971651
36,K-Means,"[2, 2, 4, 2, 2, 2, 3, 3, 2, 2, 4, 4, 2, 2, 2, ...",U,5.0,1193,cl5,0.976411,0.958415
37,K-Means,"[3, 3, 1, 3, 3, 3, 0, 0, 3, 3, 1, 1, 3, 3, 3, ...",S,5.0,1193,cl5,0.989094,0.979287
38,K-Means,"[0, 0, 3, 0, 0, 0, 4, 4, 0, 0, 3, 3, 0, 0, 0, ...",U+S,5.0,1193,cl5,0.991180,0.982610
39,K-Means,"[0, 0, 2, 0, 0, 0, 4, 4, 0, 0, 2, 2, 0, 0, 0, ...",UcatS,5.0,1193,cl5,0.989056,0.979051
56,K-Means,"[9, 7, 7, 2, 0, 10, 3, 7, 0, 2, 10, 10, 10, 0,...",U,11.0,359,allen_b02h01a02,0.621291,0.800626
57,K-Means,"[5, 10, 10, 0, 4, 6, 7, 10, 6, 0, 1, 2, 1, 4, ...",S,11.0,359,allen_b02h01a02,0.568339,0.694427


In [23]:
unpickled_df_LK[unpickled_df_LK.Method.isin(['Leiden']) & unpickled_df_LK.Hyperparam.isin([1.00]) ]

,Method,Clustering,Matrices,Hyperparam,HVGs,Data,ARI,AMI
4,Leiden,"[3, 3, 1, 3, 3, 3, 4, 4, 3, 3, 1, 1, 3, 3, 3, ...",U,1.0,357,cl5,0.948656,0.916597
5,Leiden,"[3, 3, 1, 3, 3, 3, 5, 5, 3, 3, 1, 1, 3, 3, 3, ...",S,1.0,357,cl5,0.846898,0.900251
6,Leiden,"[3, 3, 1, 3, 3, 3, 5, 5, 3, 3, 1, 1, 3, 3, 3, ...",U+S,1.0,357,cl5,0.833441,0.893414
7,Leiden,"[3, 3, 1, 3, 3, 3, 5, 5, 3, 3, 1, 1, 3, 3, 3, ...",UcatS,1.0,357,cl5,0.841939,0.893758
24,Leiden,"[3, 3, 1, 3, 3, 3, 4, 4, 3, 3, 1, 1, 3, 3, 3, ...",U,1.0,1193,cl5,0.939578,0.906968
25,Leiden,"[3, 3, 1, 3, 3, 3, 5, 5, 3, 3, 1, 1, 3, 3, 3, ...",S,1.0,1193,cl5,0.839479,0.883125
26,Leiden,"[3, 3, 2, 3, 3, 3, 5, 5, 3, 3, 2, 2, 3, 3, 3, ...",U+S,1.0,1193,cl5,0.769814,0.865851
27,Leiden,"[3, 3, 1, 3, 3, 3, 4, 4, 3, 3, 1, 1, 3, 3, 3, ...",UcatS,1.0,1193,cl5,0.851972,0.903597
44,Leiden,"[10, 1, 1, 5, 2, 3, 8, 1, 2, 5, 3, 0, 0, 2, 1,...",U,1.0,359,allen_b02h01a02,0.654273,0.823200
45,Leiden,"[6, 0, 0, 4, 3, 1, 5, 0, 0, 4, 1, 2, 1, 3, 0, ...",S,1.0,359,allen_b02h01a02,0.554039,0.646380


In [14]:
unpickled_df_LK[unpickled_df_LK.Method.isin(['Leiden']) & unpickled_df_LK.Hyperparam.isin([0.75]) ]


,Method,Clustering,Matrices,Hyperparam,HVGs,Data,ARI,AMI
0,Leiden,"[3, 3, 1, 3, 3, 3, 4, 4, 3, 3, 1, 1, 3, 3, 3, ...",U,0.75,357,cl5,0.948212,9.156829e-01
1,Leiden,"[3, 3, 1, 3, 3, 3, 4, 4, 3, 3, 1, 1, 3, 3, 3, ...",S,0.75,357,cl5,0.986832,9.757465e-01
2,Leiden,"[3, 3, 1, 3, 3, 3, 4, 4, 3, 3, 1, 1, 3, 3, 3, ...",U+S,0.75,357,cl5,0.965742,9.467628e-01
3,Leiden,"[3, 3, 1, 3, 3, 3, 4, 4, 3, 3, 1, 1, 3, 3, 3, ...",UcatS,0.75,357,cl5,0.965702,9.470799e-01
20,Leiden,"[3, 3, 1, 3, 3, 3, 4, 4, 3, 3, 1, 1, 3, 3, 3, ...",U,0.75,1193,cl5,0.941836,9.101720e-01
21,Leiden,"[3, 3, 1, 3, 3, 3, 4, 4, 3, 3, 1, 1, 3, 3, 3, ...",S,0.75,1193,cl5,0.970077,9.544757e-01
22,Leiden,"[3, 3, 1, 3, 3, 3, 4, 4, 3, 3, 1, 1, 3, 3, 3, ...",U+S,0.75,1193,cl5,0.970676,9.540767e-01
23,Leiden,"[3, 3, 1, 3, 3, 3, 4, 4, 3, 3, 1, 1, 3, 3, 3, ...",UcatS,0.75,1193,cl5,0.977565,9.625895e-01
40,Leiden,"[9, 0, 0, 5, 3, 2, 7, 0, 3, 5, 2, 1, 1, 3, 0, ...",U,0.75,359,allen_b02h01a02,0.729684,8.556821e-01
41,Leiden,"[3, 0, 0, 3, 0, 1, 4, 0, 0, 3, 1, 2, 1, 4, 0, ...",S,0.75,359,allen_b02h01a02,0.517760,5.975224e-01


### **Run Methods on Simulated Data**

In [232]:
d_strings  = glob.glob('/home/tchari/counts/meKMeans_sims/loom/*.loom')
d_strings 

['/home/tchari/counts/meKMeans_sims/loom/drop_1perc_sim_k8.loom',
 '/home/tchari/counts/meKMeans_sims/loom/drop_5perc_sim_k8.loom',
 '/home/tchari/counts/meKMeans_sims/loom/drop_20perc_sim_k8.loom',
 '/home/tchari/counts/meKMeans_sims/loom/negctrl_sim_k1.loom',
 '/home/tchari/counts/meKMeans_sims/loom/b_beta_sim_k8.loom',
 '/home/tchari/counts/meKMeans_sims/loom/k_gamma_sim_k8.loom']

In [233]:
#Not doing dropout data for now
sub_d_strings = d_strings[3:]
sub_d_strings

['/home/tchari/counts/meKMeans_sims/loom/negctrl_sim_k1.loom',
 '/home/tchari/counts/meKMeans_sims/loom/b_beta_sim_k8.loom',
 '/home/tchari/counts/meKMeans_sims/loom/k_gamma_sim_k8.loom']

In [234]:
attribute_names =[('unspliced','spliced'),'gene_name','barcode']

In [235]:
datas = ['negctrl_sim_k1','b_beta_sim_k8','k_gamma_sim_k8']

In [236]:
samp = [-6.777777777777778, -1.25]

In [237]:
#Run Leiden + K-Means first, with K=Leiden # of clus for neg control

#Make the U,S, U+S, Concat matrices, res 0.75,1,1.5,2

!mkdir /home/tchari/perturbCME/notebooks/fits/LeidKMeansruns_0306


#Save method, clustering, mat_in (U,S, etc), hyper (K or res), hvgs, data, ARI, AMI
LK_res = pd.DataFrame()
meths = []
result = []
mats = []
hyper = []
hvgs = []
data = []
ari = []
ami = []

for d in sub_d_strings:
    
    #Get tech param by keyword (datas) in loom paths
    ind = [i in d for i in datas]
    

    data_name = np.array(datas)[ind][0]
    
    
    ds = lp.connect(d)
    num_genes = len(ds.ra[attribute_names[1]])
    num_cells = len(ds.ca[attribute_names[2]])
    U = ds.layers[attribute_names[0][0]][:,:].T
    S = ds.layers[attribute_names[0][1]][:,:].T
    true_labs = list(ds.ca['subclass_label'])
    ds.close()
    print(data_name+' with '+str(num_genes)+' HVGs')
    
    
    res = [1]
    
    #Get U,S,U_p_S,U_c_S
        
    #Norm mats
    U_norm = norm_mat(U)
    S_norm = norm_mat(S)
    U_p_S_norm = norm_mat(U+S)
    U_c_S_norm = norm_mat(np.concatenate([U,S],axis=1))
    
    g_U = get_graph(U_norm)
    g_S = get_graph(S_norm)
    g_U_p_S = get_graph(U_p_S_norm)
    g_U_c_S = get_graph(U_c_S_norm)
    
    # -------- Fit Leiden -------
    for r in res:
        
        partition_type = la.RBConfigurationVertexPartition
        #Make Knn graphs
        for g in [g_U,g_S,g_U_p_S,g_U_c_S]:
            part = la.find_partition(g, partition_type, 
                                        weights=np.array(g.es['weight']).astype(np.float64), 
                                        n_iterations=-1, resolution_parameter=r)

            labels = np.array(part.membership)
            
            # ----- Save Results ---- ARI,AMI etc
            meths += ['Leiden']
            result += [labels]
            hyper += [r]
            hvgs += [num_genes]
            data += [data_name]
            ari += [adjusted_rand_score(true_labs,labels)]
            ami += [adjusted_mutual_info_score(true_labs,labels)]
            
#             print('No. clus Leiden: ',len(np.unique(labels)))
            
        mats += ['U','S','U+S','UcatS']
        
        
        
    # -------- Fit K-Means with k -------
    if 'negctrl' in d:
        k = len(np.unique(labels)) #**************** FILL THIS IN ****************
        print('Negctrl k: ',k)
    else:
        k = len(np.unique(true_labs))
        
    
    for x in [U_norm,S_norm,U_p_S_norm,U_c_S_norm]:
        kmeans = KMeans(n_clusters=k, random_state=0).fit(x)
        labs = kmeans.labels_
        
        meths += ['K-Means']
        result += [labs]
        hyper += [k]
        hvgs += [num_genes]
        data += [data_name]
        ari += [adjusted_rand_score(true_labs,labs)]
        ami += [adjusted_mutual_info_score(true_labs,labs)]
        
#         print('No. clus K-Means: ',len(np.unique(labs)))
        
    mats += ['U','S','U+S','UcatS']

    
LK_res['Method'] = meths
LK_res['Clustering'] = result
LK_res['Matrices'] = mats
LK_res['Hyperparam'] = hyper
LK_res['HVGs'] = hvgs
LK_res['Data'] = data
LK_res['ARI'] = ari
LK_res['AMI'] = ami

mkdir: cannot create directory ‘/home/tchari/perturbCME/notebooks/fits/LeidKMeansruns_0306’: File exists
negctrl_sim_k1 with 1100 HVGs
Negctrl k:  10
b_beta_sim_k8 with 1100 HVGs
k_gamma_sim_k8 with 700 HVGs


In [208]:
#RERUN K-Means = 10

In [238]:
LK_res

,Method,Clustering,Matrices,Hyperparam,HVGs,Data,ARI,AMI
0,Leiden,"[0, 1, 7, 8, 2, 0, 2, 1, 0, 3, 0, 7, 0, 5, 4, ...",U,1,1100,negctrl_sim_k1,0.000000,1.837018e-16
1,Leiden,"[14, 12, 13, 12, 2, 13, 3, 11, 2, 13, 3, 14, 0...",S,1,1100,negctrl_sim_k1,0.000000,2.178137e-16
2,Leiden,"[5, 9, 12, 3, 2, 9, 12, 0, 3, 5, 0, 9, 10, 1, ...",U+S,1,1100,negctrl_sim_k1,0.000000,2.179588e-16
3,Leiden,"[1, 5, 1, 1, 5, 1, 8, 0, 5, 4, 3, 3, 6, 1, 0, ...",UcatS,1,1100,negctrl_sim_k1,0.000000,6.054004e-16
4,K-Means,"[4, 4, 3, 2, 8, 8, 0, 9, 6, 5, 4, 9, 6, 8, 2, ...",U,10,1100,negctrl_sim_k1,0.000000,2.613066e-16
5,K-Means,"[8, 5, 0, 8, 8, 5, 1, 4, 9, 9, 2, 3, 4, 1, 1, ...",S,10,1100,negctrl_sim_k1,0.000000,1.389040e-16
6,K-Means,"[4, 7, 2, 2, 4, 9, 5, 1, 6, 7, 4, 9, 8, 6, 2, ...",U+S,10,1100,negctrl_sim_k1,0.000000,2.272039e-17
7,K-Means,"[8, 3, 7, 0, 5, 5, 0, 4, 0, 1, 1, 7, 5, 5, 9, ...",UcatS,10,1100,negctrl_sim_k1,0.000000,-1.629062e-16
8,Leiden,"[6, 0, 6, 6, 6, 6, 6, 6, 0, 6, 6, 6, 0, 0, 6, ...",U,1,1100,b_beta_sim_k8,0.743029,8.097669e-01
9,Leiden,"[2, 4, 3, 4, 8, 9, 0, 3, 3, 7, 5, 3, 3, 2, 4, ...",S,1,1100,b_beta_sim_k8,0.029161,3.302685e-02


In [239]:
LK_res.to_pickle('./fits/LeidKMeansruns_0306/LK_sim_results.pkl') #0301
#unpickled_df = pd.read_pickle("./meK_res.pkl")

In [241]:
transcriptomes = ['/home/tchari/perturbCME/notebooks/sim_genome_polyA_cum_3_0',
                  '/home/tchari/perturbCME/notebooks/sim_genome_polyA_cum_3_0',
                 '/home/tchari/perturbCME/notebooks/sim_genome_polyA_cum_3_1']

In [243]:
#For each loom in looms from ./hvg_objs

# !mkdir /home/tchari/perturbCME/notebooks/fits/meKruns_0209
!mkdir /home/tchari/perturbCME/notebooks/fits/meKruns_0306


dir_strings = []
data_strings = []
result_strings = []

#Save method, clustering, mat_in (U,S, etc), hyper (K or res), hvgs, data, ARI, AMI
meK_res = pd.DataFrame()
meths = []
result = []
mats = []
hyper = []
hvgs = []
data = []
ari = []
ami = []

for d in sub_d_strings[1:]: #************* FASTER FOR NOW ************* 
    
    #Get tech param by keyword (datas) in loom paths
    ind = [i in d for i in datas]
    
    #Inference params for sampling
    tech_tup = samp
    
    #Preprocessing params for filepaths
    trans = np.array(transcriptomes)[ind][0]

    data_name = np.array(datas)[ind][0]
    
    
    ds = lp.connect(d)
    g_names = ds.ra[attribute_names[1]]
    num_genes = len(g_names)
    num_cells = len(ds.ca[attribute_names[2]])
    
    U = ds.layers['unspliced'][:,:].T
    S = ds.layers['spliced'][:,:].T
    true_labs = list(ds.ca['subclass_label'])
    ds.close()

    
    print(data_name+' with '+str(num_genes)+' genes')
    
    
    if 'negctrl' in d:
        Ks = [10] #***FILL THIS IN from leiden *******
    else:
        Ks = [len(np.unique(true_labs))]
        

    for clus in Ks:
       
        #Run Monod preprocessing to set up files for run, leave room for genes to be removed with low U/S
        dir_string,dataset_string = monod.preprocess.construct_batch([d], \
                                                     trans, \
                                                     [data_name], \
                                                     attribute_names=attribute_names,\
                                                     batch_location='./fits/meKruns_0306', \
                                                     meta=data_name+'_K'+str(clus), \
                                                     batch_id=1, \
                                                     n_genes=num_genes,exp_filter_threshold=None,
                                                                     viz=False,genes_to_fit=g_names)
                                                        #num_genes-60 or 100,genes_to_fit=markers
        dir_strings += [dir_string]
        data_strings += [dataset_string]

        #Set model and run meK-Means
        
        #Define bounds for params
        phys_lb = [-3.0, -3, -3 ] 
        phys_ub = [4.2, 2.5, 2.5] 
        samp_lb = tech_tup 
        samp_ub = tech_tup  
        gridsize = [1,1] #Already have tech params
    

        epochs = 10

        # ---------------- meK-Means Inference ---------------- 
        #Define model with bursty transcription and Poisson molecule capture/sampling
        fitmodel = monod.cme_toolbox.CMEModel('Bursty','Poisson')

        #Set up mminference parameters
        inference_parameters = monod.mminference.InferenceParameters(phys_lb,phys_ub,samp_lb,samp_ub,gridsize,\
                    dataset_string[0],fitmodel,k=clus,epochs=epochs,use_lengths = True,
                    gradient_params = {'max_iterations':5,'init_pattern':'moments','num_restarts':1})

        #Read in loom file with filtered barcodes
        search_data = monod.extract_data.extract_data(d, trans, data_name,
                    dataset_string[0], dir_string, viz=False, dataset_attr_names=attribute_names)

        #Run inference(fit_all_grid_points()) and Save result file strings
        full_result_string = inference_parameters.fit_all_grid_points(40,search_data) 

        result_strings.append(full_result_string)
        

        # ----------------  Save output ---------------- 
        #Read in results and get cluster assignments
        sr = [monod.analysis.load_search_results(i) for i in full_result_string]
        
        
        cat_assigns = np.array([None]*num_cells) #cat_assigns is empty list of length = no. cell barcodes
        for i in range(len(sr)):
            r = sr[i]
            cat_assigns[r.filt] = r.assigns #denotes which cluster (k), cells were assigned to

        print('No. clus: ',len(np.unique(cat_assigns)))
        

        meths += ['meK-Means']
        result += [cat_assigns]
        mats += ['U,S']
        hyper += [clus]
        hvgs += [num_genes]
        data += [data_name]
        ari += [adjusted_rand_score(true_labs,cat_assigns)]
        ami += [adjusted_mutual_info_score(true_labs,cat_assigns)]
        
        print('ARI: ',adjusted_rand_score(true_labs,cat_assigns))


meK_res['Method'] = meths
meK_res['Clustering'] = result
meK_res['Matrices'] = mats
meK_res['Hyperparam'] = hyper
meK_res['Genes'] = hvgs
meK_res['Data'] = data
meK_res['ARI'] = ari
meK_res['AMI'] = ami
    

mkdir: cannot create directory ‘/home/tchari/perturbCME/notebooks/fits/meKruns_0306’: File exists
b_beta_sim_k8 with 1100 genes


100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [01:50<00:00, 11.03s/it]


mstep self.weights:  [0.02130789 0.21951494 0.16482509 0.1216082  0.06894709 0.08862405
 0.06587888 0.07057967 0.12444889 0.0542653 ]
Q Function:  -7646860.875121265



100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [01:41<00:00, 10.14s/it]


mstep self.weights:  [0.00069093 0.23858718 0.19537353 0.15939084 0.05120111 0.0600907
 0.09508959 0.07557237 0.10175058 0.02225315]
Q Function:  -7598969.520596657



...

mstep self.weights:  [0.11784623 0.29419278 0.06957437 0.37160211 0.14678452]
Q Function:  -3334269.140620426



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:41<00:00,  8.33s/it]


mstep self.weights:  [0.11784626 0.29419278 0.01628122 0.42489522 0.14678452]
Q Function:  -3336645.365907383



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:40<00:00, 10.21s/it]


mstep self.weights:  [1.17846235e-01 2.94192768e-01 1.94523151e-08 4.41176455e-01
 1.46784523e-01]
Q Function:  -3335875.9705061186

No. clus:  4
ARI:  0.9875836314870531


In [244]:
meK_res

,Method,Clustering,Matrices,Hyperparam,Genes,Data,ARI,AMI
0,meK-Means,"[7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, ...","U,S",10,1100,b_beta_sim_k8,0.977105,0.957871
1,meK-Means,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","U,S",5,700,k_gamma_sim_k8,0.987584,0.972653


In [245]:
meK_res.to_pickle('./fits/meKruns_0306/meK_sim_results.pkl') #0301
#unpickled_df = pd.read_pickle("./meK_res.pkl")

In [249]:
np.array(transcriptomes)[0]

'/home/tchari/perturbCME/notebooks/sim_genome_polyA_cum_3_0'

In [250]:
for d in [sub_d_strings[0]]: #Forgot to run negctrl
    
    #Get tech param by keyword (datas) in loom paths
    ind = [i in d for i in datas]
    
    #Inference params for sampling
    tech_tup = samp
    
    #Preprocessing params for filepaths
    trans = np.array(transcriptomes)[ind][0]

    data_name = np.array(datas)[ind][0]
    
    
    ds = lp.connect(d)
    g_names = ds.ra[attribute_names[1]]
    num_genes = len(g_names)
    num_cells = len(ds.ca[attribute_names[2]])
    
    U = ds.layers['unspliced'][:,:].T
    S = ds.layers['spliced'][:,:].T
    true_labs = list(ds.ca['subclass_label'])
    ds.close()

    
    print(data_name+' with '+str(num_genes)+' genes')
    
    
    if 'negctrl' in d:
        Ks = [10] #***FILL THIS IN from leiden *******
    else:
        Ks = [len(np.unique(true_labs))]
        

    for clus in Ks:
       
        #Run Monod preprocessing to set up files for run, leave room for genes to be removed with low U/S
        dir_string,dataset_string = monod.preprocess.construct_batch([d], \
                                                     trans, \
                                                     [data_name], \
                                                     attribute_names=attribute_names,\
                                                     batch_location='./fits/meKruns_0306', \
                                                     meta=data_name+'_K'+str(clus), \
                                                     batch_id=1, \
                                                     n_genes=num_genes,exp_filter_threshold=None,
                                                                     viz=False,genes_to_fit=g_names)
                                                        #num_genes-60 or 100,genes_to_fit=markers
        dir_strings += [dir_string]
        data_strings += [dataset_string]

        #Set model and run meK-Means
        
        #Define bounds for params
        phys_lb = [-3.0, -3, -3 ] 
        phys_ub = [4.2, 2.5, 2.5] 
        samp_lb = tech_tup 
        samp_ub = tech_tup  
        gridsize = [1,1] #Already have tech params
    

        epochs = 10

        # ---------------- meK-Means Inference ---------------- 
        #Define model with bursty transcription and Poisson molecule capture/sampling
        fitmodel = monod.cme_toolbox.CMEModel('Bursty','Poisson')

        #Set up mminference parameters
        inference_parameters = monod.mminference.InferenceParameters(phys_lb,phys_ub,samp_lb,samp_ub,gridsize,\
                    dataset_string[0],fitmodel,k=clus,epochs=epochs,use_lengths = True,
                    gradient_params = {'max_iterations':5,'init_pattern':'moments','num_restarts':1})

        #Read in loom file with filtered barcodes
        search_data = monod.extract_data.extract_data(d, trans, data_name,
                    dataset_string[0], dir_string, viz=False, dataset_attr_names=attribute_names)

        #Run inference(fit_all_grid_points()) and Save result file strings
        full_result_string = inference_parameters.fit_all_grid_points(40,search_data) 

        result_strings.append(full_result_string)
        

        # ----------------  Save output ---------------- 
        #Read in results and get cluster assignments
        sr = [monod.analysis.load_search_results(i) for i in full_result_string]
        
        
        cat_assigns = np.array([None]*num_cells) #cat_assigns is empty list of length = no. cell barcodes
        for i in range(len(sr)):
            r = sr[i]
            cat_assigns[r.filt] = r.assigns #denotes which cluster (k), cells were assigned to

        print('No. clus: ',len(np.unique(cat_assigns)))
        

        meths += ['meK-Means']
        result += [cat_assigns]
        mats += ['U,S']
        hyper += [clus]
        hvgs += [num_genes]
        data += [data_name]
        ari += [adjusted_rand_score(true_labs,cat_assigns)]
        ami += [adjusted_mutual_info_score(true_labs,cat_assigns)]
        
        print('ARI: ',adjusted_rand_score(true_labs,cat_assigns))


negctrl_sim_k1 with 1100 genes


100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [01:38<00:00,  9.81s/it]


mstep self.weights:  [0.03866153 0.27886566 0.10848511 0.02935103 0.02319026 0.21954071
 0.08938233 0.07991915 0.01599268 0.11661154]
Q Function:  -7205402.919601542



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [01:40<00:00, 11.22s/it]


mstep self.weights:  [3.42013288e-03 4.91752303e-01 1.30272183e-01 9.84453936e-04
 6.28162594e-06 1.48956136e-01 4.33010212e-02 4.83604226e-02
 8.84346411e-04 1.32062719e-01]
Q Function:  -7195788.6708520325



...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [01:29<00:00, 89.51s/it]


mstep self.weights:  [2.00000000e-10 9.99999998e-01 2.00000000e-10 2.00000000e-10
 2.00027670e-10 2.00000000e-10 2.00000000e-10 2.00000000e-10
 2.00000000e-10 2.00000007e-10]
Q Function:  -7189215.459900708



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [01:35<00:00, 95.31s/it]


mstep self.weights:  [2.00000000e-10 9.99999998e-01 2.00000000e-10 2.00000000e-10
 2.00027670e-10 2.00000000e-10 2.00000000e-10 2.00000000e-10
 2.00000000e-10 2.00000007e-10]
Q Function:  -7189215.459900708

No. clus:  1
ARI:  1.0


ValueError: Length of values (3) does not match length of index (2)

In [252]:
meK_res = pd.DataFrame()

meK_res['Method'] = meths
meK_res['Clustering'] = result
meK_res['Matrices'] = mats
meK_res['Hyperparam'] = hyper
meK_res['Genes'] = hvgs
meK_res['Data'] = data
meK_res['ARI'] = ari
meK_res['AMI'] = ami

In [253]:
meK_res

,Method,Clustering,Matrices,Hyperparam,Genes,Data,ARI,AMI
0,meK-Means,"[7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, ...","U,S",10,1100,b_beta_sim_k8,0.977105,0.957871
1,meK-Means,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","U,S",5,700,k_gamma_sim_k8,0.987584,0.972653
2,meK-Means,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","U,S",10,1100,negctrl_sim_k1,1.000000,1.000000


In [254]:
meK_res.to_pickle('./fits/meKruns_0306/meK_sim_results.pkl') #0301
#unpickled_df = pd.read_pickle("./meK_res.pkl")